In [13]:
import pandas as pd
import numpy as np

# 删除异常数据点
def delete_outliers(data, point_name):
    # print(data.isna().sum())
    if point_name in ['回风温度', '送风温度', '水路进水温度']:
        data[(data<0)|(data>60)] = np.nan
    if point_name == '风机转速':
        data[(data<0)|(data>2000)] = np.nan
    if point_name == '水阀开度':
        data[(data<0)|(data>100)] = np.nan   
    if point_name == '冷通道温度':
        data[(data<0.5)|(data>80)] = np.nan
    # if point_name in ['系统故障状态', '风机运行状态', '制冷状态', '远程关机', '系统运行状态']:
    #     data[(data!=0)|(data!=1)] = np.nan
    # print(data.isna().sum())
    return data

# 预处理，补充缺失的时间点，然后再对缺失的时间点用前面或者后面的值填充
def get_timeseries(df, time_col):
    df[time_col] = pd.to_datetime(df[time_col])
    ts_series = df[time_col].drop_duplicates()
    ts_series.sort_values(inplace=True)
    return ts_series


def get_freq(ts_series):
    """
    :param ts_series(series):
    :return:
    """
    ts_diff = ts_series.diff().dropna()
    ts_most_appear = int(ts_diff.value_counts().index[0].total_seconds())  # in seconds
    return ts_most_appear
    # return "{}s".format(ts_most_appear)


def get_real_index(df, time_col):
    """
    :param df:
    :param time_col:
    :return:
    """
    ts_series = get_timeseries(df, time_col)
    f_int = get_freq(ts_series)
    f = "{}s".format(f_int)
    st, ed = ts_series.min(), ts_series.max()
    real_index = pd.date_range(st, ed, freq=f)
    return real_index


def add_time_idx(df, time_col):
    assert len(df) != 0, "pandas data length is zero"
    assert time_col is not None, "time col should be given"
    real_time = get_real_index(df, time_col)
    df_real_time = pd.DataFrame(real_time, columns=[time_col])
    df_real_time.reset_index(inplace=True)
    df_real_time.rename(columns={"index": "time_idx"}, inplace=True)
    df_real_time["time_idx"] = df_real_time.time_idx.astype("int")
    df_format = pd.merge(df, df_real_time, on='collect_time', how="right")
    return df_format

def random_dates(start, end, n=1):
    start_u = start.value//10**9
    print(start_u)
    end_u = end.value//10**9
    return pd.to_datetime(np.random.randint(start_u, end_u, n), unit='s')

In [14]:
def data_preprocess(train_data):

    # delete outliers
    for i in range(train_data.columns.size-1):
        column_name = train_data.columns[i+1]
        if column_name[len(column_name)-3:] == '001' or column_name[len(column_name)-3:] == '003' :
            point_name = column_name[:len(column_name)-3]
        else:
            point_name = column_name[8:]
        # print(point_name)
        train_data[column_name] = delete_outliers(train_data[column_name],point_name)

    # forward and backward fill 前向后向填充
    # print('nan before fill:\n',train_data.isna().sum())
    data_fill_f = train_data.fillna(method='ffill')
    data_fill = data_fill_f.fillna(method='bfill')
    # print('nan after fill:\n',data_fill.isna().sum(),data_fill.shape)
    
    return data_fill

In [15]:
import os
import pandas as pd
import numpy as np

def trainlabel2npy(i_interv, load_path, save_path, filename, init_order):
    abnormal_data = pd.read_csv(load_path+'abnormal/'+filename)
    node_name = pd.DataFrame(abnormal_data.columns.values[1:])
    node_name.to_csv(save_path+'node_name.csv')
    abnormal_data = data_preprocess(abnormal_data)
    print(abnormal_data)
    abnormal_data_np = np.array(abnormal_data.iloc[:,1:])
    num_time = abnormal_data_np.shape[0]
    np.save(save_path+f'abnormal/{i_interv}_interv.npy',abnormal_data_np)

    filelist = os.listdir(load_path+'/normal/')
    j = 1
    for file in filelist:
        filename = str(file)
        if filename[len(filename)-4:] == '.csv':
            normal_data = pd.read_csv(load_path+'/normal/'+filename)
            normal_data = data_preprocess(normal_data)
            normal_data_np = np.array(normal_data.iloc[:,1:])
            np.save(save_path+f'normal/{j}_normal.npy',normal_data_np)
            j = j+1

    label2csv(i_interv, load_path, save_path, init_order, num_time, node_name)


def label2csv(i_interv, load_path, save_path, init_order, num_time, node_name):
    filename = str(f'{init_order}_label.csv') 
    label = pd.read_csv(load_path +'abnormal/' + filename).iloc[:,1]
    label = label.values.flatten().tolist()
    # change label to int
    for i in range(len(label)):
        for j in range(node_name.shape[0]):
            if label[i] == node_name.iloc[j,0]:
                label[i] = j
                break
    label_data = pd.DataFrame([label]*num_time)
    label_data.to_csv(save_path+'abnormal/'+f'{i_interv}_label.csv',index=False,header=False)

def change_train_label(load_path, save_path):
    os.makedirs(save_path+'abnormal',exist_ok=True)
    os.makedirs(save_path+'normal',exist_ok=True)
    filelist = os.listdir(load_path+'abnormal/')
    i_interv = 1
    correspond_list = []
    for file in filelist:
        filename = str(file)
        init_order = filename[:len(filename)-10]
        if filename[len(filename)-10:] == '_train.csv':
            trainlabel2npy(i_interv, load_path, save_path, filename, init_order)
            correspond_list.append([i_interv,init_order])
            i_interv += 1
    correspond_list = pd.DataFrame(correspond_list,columns=['i_dataset','init_num'])
    correspond_list.to_csv(save_path+'correspond_list.csv',index=False)

In [16]:
change_train_label(load_path = f'./raw/upstream_less3/', 
                   save_path = f'./transfered/upstream_less3/') # upstream
# change_train_label(load_path = f'./raw/downstream_less/', 
#                    save_path = f'./transfered/downstream_less/') # downstream

              Unnamed: 0  2149056_回风温度001  2149056_水路进水温度003  2149056_送风温度001  \
0    2023-05-05 13:56:30        36.100002               20.0             24.4   
1    2023-05-05 13:56:40        36.100002               20.0             24.4   
2    2023-05-05 13:56:50        36.100002               20.0             24.4   
3    2023-05-05 13:57:00        36.100002               20.0             24.4   
4    2023-05-05 13:57:10        36.100002               20.0             24.4   
..                   ...              ...                ...              ...   
115  2023-05-05 14:15:40        36.200000               19.9             24.4   
116  2023-05-05 14:15:50        36.200000               19.9             24.4   
117  2023-05-05 14:16:00        36.200000               19.9             24.4   
118  2023-05-05 14:16:10        36.200000               19.9             24.4   
119  2023-05-05 14:16:20        36.200000               19.9             24.4   

     2149057_回风温度001  21490